In [ ]:
! pip install Flask


In [ ]:
! pip install import_ipynb

In [ ]:
! pip install pymysql

In [ ]:
! pip install WTForms

In [ ]:
! pip install flask-login

In [ ]:
! pip freeze

In [ ]:
from flask import Flask, request, render_template, redirect, url_for, jsonify
import import_ipynb
import forms

In [ ]:
# intilize flask application
app = Flask(__name__)

In [ ]:
@app.route("/")
def home():
    return redirect(url_for("login"))

In [ ]:
@app.route("/login",methods=["GET","POST"])
def login():
    if request.method=="POST":
        pass
    else :
        return render_template("auth/user.html")

In [ ]:
@app.route("/register",methods=["GET","POST"])
def register():
    if request.method=="POST":
        print(request.form)
        register_form=forms.RegistrationForm(request.form)
        print(register_form.data)
        if register_form.validate():
            return jsonify(register_form.data)
        else :
            return jsonify({"errors" : register_form.errors }), 400
    else :
        return render_template("auth/registartion.html")



In [ ]:
if __name__== "__main__":
    app.run(debug=False)